In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [2]:

raw_df = spark.read \
.format("csv") \
.option("inferSchema", "true") \
.option("header", "true") \
.load("/public/trendytech/datasets/accepted_2007_to_2018Q4.csv")

In [3]:
raw_df.createOrReplaceTempView("lending_club_data")

In [4]:
spark.sql("select * from lending_club_data")

+--------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+-----------+----------+--------------------+----+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+------------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--------------+-------+------------------------+-----------+------------------+--------------------+---------------------+--------------+--------+--------------------+------------------------+---------------------+------------------------------+---------------------+--------------+---------------+-----------+---------+---------+-------------+-------------+-------------------+--------+----------------+------------+------------------+------------------+--------------+----------------+--------------------+---------+---------------+-----------------+--------------+--------------------------+---------------+----------------------+-----------------------+------------------------+----------------------+----------------+----------------+------------------+-------------------+---------------------+--------------------------------+----------------------------------+-----------------------------------+-------------+-------------+---------------+---------------+-------------+---------------+-------------------+-----------------+-----------------------+---------------+------------+--------------------+------------------------------------------+------------------------------+----------------------------+-------------------+--------------------+-------------------------+-----------------+---------------+-----------------+---------------------+---------------+
|      id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|      term|int_rate|installment|grade|sub_grade|           emp_title|emp_length|home_ownership|annual_inc|verification_status| issue_d|loan_status|pymnt_plan|                 url|desc|           purpose|               title|zip_code|addr_state|  dti|delinq_2yrs|earliest_cr_line|fico_range_low|fico_range_high|inq_last_6mths|mths_since_last_delinq|mths_since_last_record|open_acc|pub_rec|revol_bal|revol_util|total_acc|initial_list_status|out_prncp|out_prncp_inv|       total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|last_pymnt_d|last_pymnt_amnt|next_pymnt_d|last_credit_pull_d|last_fico_range_high|last_fico_range_low|collections_12_mths_ex_med|mths_since_last_major_derog|policy_code|application_type|annual_inc_joint|dti_joint|verification_status_joint|acc_now_delinq|tot_coll_amt|tot_cur_bal|open_acc_6m|open_act_il|open_il_12m|open_il_24m|mths_since_rcnt_il|total_bal_il|il_util|open_rv_12m|open_rv_24m|max_bal_bc|all_util|total_rev_hi_lim|inq_fi|total_cu_tl|inq_last_12m|acc_open_past_24mths|avg_cur_bal|bc_open_to_buy|bc_util|chargeoff_within_12_mths|delinq_amnt|mo_sin_old_il_acct|mo_sin_old_rev_tl_op|mo_sin_rcnt_rev_tl_op|mo_sin_rcnt_tl|mort_acc|mths_since_recent_bc|mths_since_recent_bc_dlq|mths_since_recent_inq|mths_since_recent_revol_delinq|num_accts_ever_120_pd|num_actv_bc_tl|num_actv_rev_tl|num_bc_sats|num_bc_tl|num_il_tl|num_op_rev_tl|num_rev_accts|num_rev_t

#### Import sha2

In [5]:
from pyspark.sql.functions import sha2, concat_ws

In [6]:

new_df = raw_df.withColumn("name_sha2",sha2(concat_ws("||",*["emp_title","emp_length","home_ownership","annual_inc","zip_code","addr_state","grade","sub_grade","verification_status"]),256))

In [7]:
new_df.createOrReplaceTempView("newtable")

In [8]:
spark.sql("select count(*) from newtable")

+--------+
|count(1)|
+--------+
| 2260701|
+--------+

In [9]:
spark.sql("select count(distinct(name_sha2)) from newtable")

+-------------------------+
|count(DISTINCT name_sha2)|
+-------------------------+
|                  2257384|
+-------------------------+

#### See the top occuring member_ids

In [10]:
spark.sql("""select name_sha2, count(*) as total_cnt from newtable 
group by name_sha2 
having total_cnt > 1
order by total_cnt desc""")

+--------------------+---------+
|           name_sha2|total_cnt|
+--------------------+---------+
|e3b0c44298fc1c149...|       33|
|e4c167053d5418230...|        5|
|3f87585a20f702838...|        4|
|ad8e5d384dae17e06...|        4|
|76b577467eda5bdbc...|        4|
|c92062bb371842b3d...|        3|
|059d401bb603d9a80...|        3|
|d93573e2883e37904...|        3|
|066ddaa64bee66dff...|        3|
|498bb6b1f0099cb47...|        3|
|4ab6205de571ccb7b...|        3|
|d9ce4046daf599732...|        3|
|53789bea7edc660ed...|        3|
|e7d8d16928817ec8f...|        3|
|819453be77718d747...|        3|
|a2356725aa9da92c6...|        3|
|291ca1b09ef11ca3e...|        3|
|5d52e7773cb0efff3...|        3|
|d3731bad3e6a82515...|        3|
|22593a1870543b2db...|        3|
+--------------------+---------+
only showing top 20 rows

In [11]:
spark.sql("select * from newtable where name_sha2 like 'e3b0c44298fc1c149%'")

+--------------------+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+---------+----------+--------------+----------+-------------------+-------+-----------+----------+----+----+-------+-----+--------+----------+----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--------------+-------+------------------------+-----------+------------------+--------------------+---------------------+--------------+--------+--------------------+------------------------+---------------------+------------------------------+---------------------+--------------+---------------+-----------+---------+---------+-------------+-------------+-------------------+--------+----------------+------------+------------------+------------------+--------------+----------------+--------------------+---------+---------------+-----------------+--------------+--------------------------+---------------+----------------------+-----------------------+------------------------+----------------------+----------------+----------------+------------------+-------------------+---------------------+--------------------------------+----------------------------------+-----------------------------------+-------------+-------------+---------------+---------------+-------------+---------------+-------------------+-----------------+-----------------------+---------------+------------+--------------------+------------------------------------------+------------------------------+----------------------------+-------------------+--------------------+-------------------------+-----------------+---------------+-----------------+---------------------+---------------+--------------------+
|                  id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|term|int_rate|installment|grade|sub_grade|emp_title|emp_length|home_ownership|annual_inc|verification_status|issue_d|loan_status|pymnt_plan| url|desc|purpose|title|zip_code|addr_state| dti|delinq_2yrs|earliest_cr_line|fico_range_low|fico_range_high|inq_last_6mths|mths_since_last_delinq|mths_since_last_record|open_acc|pub_rec|revol_bal|revol_util|total_acc|initial_list_status|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|last_pymnt_d|last_pymnt_amnt|next_pymnt_d|last_credit_pull_d|last_fico_range_high|last_fico_range_low|collections_12_mths_ex_med|mths_since_last_major_derog|policy_code|application_type|annual_inc_joint|dti_joint|verification_status_joint|acc_now_delinq|tot_coll_amt|tot_cur_bal|open_acc_6m|open_act_il|open_il_12m|open_il_24m|mths_since_rcnt_il|total_bal_il|il_util|open_rv_12m|open_rv_24m|max_bal_bc|all_util|total_rev_hi_lim|inq_fi|total_cu_tl|inq_last_12m|acc_open_past_24mths|avg_cur_bal|bc_open_to_buy|bc_util|chargeoff_within_12_mths|delinq_amnt|mo_sin_old_il_acct|mo_sin_old_rev_tl_op|mo_sin_rcnt_rev_tl_op|mo_sin_rcnt_tl|mort_acc|mths_since_recent_bc|mths_since_recent_bc_dlq|mths_since_recent_inq|mths_since_recent_revol_delinq|num_accts_ever_120_pd|num_actv_bc_tl|num_actv_rev_tl|num_bc_sats|num_bc_tl|num_il_tl|num_op_rev_tl|num_rev_accts|num_rev_tl_bal_gt_0|num_sats|num_tl_120dpd_2m|num_tl_30dpd|num_tl_90g_dpd_24m|num_tl_op_past_12m|pct

#### Create customers dataset

In [12]:
spark.sql("""select name_sha2 as member_id, emp_title, emp_length, home_ownership, 
annual_inc, addr_state, zip_code, 'USA' as country, grade, sub_grade,
verification_status, tot_hi_cred_lim , application_type , annual_inc_joint , verification_status_joint
from newtable""").repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","/user/itv018355/lendingclubproject/raw/customers_data_csv") \
.save()

In [13]:
#spark.stop()

In [14]:
customers_df = spark.read \
.format("csv") \
.option("inferSchema", "true") \
.option("header", "true") \
.load("/user/itv018355/lendingclubproject/raw/customers_data_csv")

In [15]:
customers_df

+--------------------+--------------------+----------+--------------+----------+----------+--------+-------+-----+---------+-------------------+---------------+----------------+----------------+-------------------------+
|           member_id|           emp_title|emp_length|home_ownership|annual_inc|addr_state|zip_code|country|grade|sub_grade|verification_status|tot_hi_cred_lim|application_type|annual_inc_joint|verification_status_joint|
+--------------------+--------------------+----------+--------------+----------+----------+--------+-------+-----+---------+-------------------+---------------+----------------+----------------+-------------------------+
|bfcec8da7b3f1fe83...|    Regional Manager|   8 years|          RENT|  130000.0|        CT|   069xx|    USA|    C|       C3|           Verified|       135904.0|      Individual|            null|                     null|
|36399458db4295868...|                null|      null|          RENT|   12000.0|        CA|   953xx|    USA|    C|       C1|           Verified|         7700.0|       Joint App|         62000.0|                 Verified|
|e4912b650878ca941...|             Manager|   6 years|      MORTGAGE|  103000.0|        CA|   922xx|    USA|    A|       A2|       Not Verified|       511251.0|      Individual|            null|                     null|
|3d6f35ad2e2be4572...|Clinical Coordinator|   6 years|      MORTGAGE|   64300.0|        NM|   871xx|    USA|    D|       D1|    Source Verified|       233646.0|      Individual|            null|                     null|
|478e1cd37e35c20f7...|Warehouse supervisor|   3 years|          RENT|   32000.0|        NJ|   073xx|    USA|    C|       C4|       Not Verified|        44200.0|      Individual|            null|                     null|
|a5d42e35cc0be2789...|       Sales Manager|   5 years|      MORTGAGE|  140000.0|        CA|   917xx|    USA|    C|       C5|    Source Verified|       455867.0|      Individual|            null|                     null|
|7f3ef55c784bf6b16...|               Owner| 10+ years|      MORTGAGE|   98000.0|        NV|   891xx|    USA|    A|       A3|    Source Verified|       265000.0|      Individual|            null|                     null|
|beb30abaeca08f49b...|Alternate Media S...|   2 years|          RENT|   56000.0|        CA|   940xx|    USA|    D|       D1|    Source Verified|        75023.0|      Individual|            null|                     null|
|233e5e6797dd0a68b...|           Hammerman|   7 years|      MORTGAGE|   70000.0|        OH|   440xx|    USA|    C|       C4|    Source Verified|       233341.0|      Individual|            null|                     null|
|c4b178ffaf80ed472...|      Civil Engineer|  < 1 year|          RENT|  120000.0|        FL|   321xx|    USA|    B|       B3|    Source Verified|        97065.0|      Individual|            null|                     null|
|7f45445bc792b29c7...|Lieutenant/Paramedic| 10+ years|      MORTGAGE|   62258.0|        MI|   488xx|    USA|    C|       C4|           Verified|       210409.0|      Individual|            null|                     null|
|582a9b2d3a658ab05...|    Shift supervisor| 10+ years|           OWN|   45000.0|        MI|   480xx|    USA|    A|       A2|    Source Verified|       119080.0|      Individual|            null|                     null|
|f67ba01e72eb5f109...|customer service rep|   4 years|      MORTGAGE|   42000.0|        OK|   741xx|    USA|    D|       D5|       Not Verified|       200965.0|      Individual|            null|                     null|
|07e81491adcc0e7c3...|               owner|   2 years|          RENT|   67350.0|        SC|   297xx|    USA|    E|       E3|    Source Verified|        74617.0|      Individual|            null|                     null|
|6f5c275cbae528914...|                null|      null|      MORTGAGE|   95000.0|        CA|   954xx|    USA|    A|       A2|       Not Verified|       504496.0|      Individual|            null|                     null|
|412a63a98577abf03...|

#### Loans dataset

In [16]:
spark.sql("""select id as loan_id , name_sha2 as member_id, loan_amnt, funded_amnt , term, int_rate, installment, issue_d, loan_status, purpose, title
from newtable""").repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","/user/itv018355/lendingclubproject/raw/loans_data_csv") \
.save()

In [17]:
loans_df = spark.read \
.format("csv") \
.option("inferSchema", "true") \
.option("header", "true") \
.load("/user/itv018355/lendingclubproject/raw/loans_data_csv")

In [18]:
loans_df

+-------+--------------------+---------+-----------+---------+--------+-----------+--------+-----------+--------------------+--------------------+
|loan_id|           member_id|loan_amnt|funded_amnt|     term|int_rate|installment| issue_d|loan_status|             purpose|               title|
+-------+--------------------+---------+-----------+---------+--------+-----------+--------+-----------+--------------------+--------------------+
| 491699|961ae110ec063761f...|   7000.0|     7000.0|36 months|   12.73|     234.97|Mar-2010| Fully Paid|  debt_consolidation|Noelle's debt con...|
| 491685|291854d244748c911...|  15000.0|    15000.0|36 months|   10.62|      488.4|Mar-2010| Fully Paid|               other|           Education|
| 491667|b5cf66b90193da9d2...|   6400.0|     6400.0|36 months|    7.88|      200.2|Mar-2010| Fully Paid|  debt_consolidation|   Make money easily|
| 491160|7f53fd1acb7759e5b...|   4000.0|     4000.0|36 months|   14.59|     137.86|Mar-2010| Fully Paid|             wedding|    wedding expenses|
| 491675|12edc8e328b993914...|  20000.0|    20000.0|36 months|   13.85|     682.08|Mar-2010| Fully Paid|               other| Short Term Tax Loan|
| 491668|e18a847f091332ff6...|   6000.0|     6000.0|36 months|   11.36|     197.47|Mar-2010|Charged Off|  debt_consolidation|GREAT BORROWER --...|
| 491663|d6fe0000e876ab76d...|   5500.0|     5500.0|36 months|   11.36|     181.02|Mar-2010| Fully Paid|         credit_card|     Road to Success|
| 491632|62c38934f3a2ebcd2...|  10000.0|    10000.0|36 months|    15.7|     350.11|Mar-2010| Fully Paid|         credit_card|Finish Paying off...|
| 491618|7701762ef5d302b0d...|  25000.0|    25000.0|36 months|   15.33|     870.71|Mar-2010| Fully Paid|  debt_consolidation|Pay off my high i...|
| 491622|42a2df45b02b21b9a...|  25000.0|    25000.0|36 months|   16.07|     879.85|Mar-2010| Fully Paid|  debt_consolidation|        Debt Pay Off|
| 491552|b1ba0adce0baea939...|  22000.0|    22000.0|36 months|   11.36|     724.05|Mar-2010| Fully Paid|  debt_consolidation|Getting rid of Ne...|
| 491477|82cdc418e3907686a...|  16750.0|    16750.0|36 months|   13.11|     565.23|Mar-2010| Fully Paid|  debt_consolidation|Special Ed teache...|
| 491596|6cbc67caa30708b24...|   3600.0|     3600.0|36 months|    15.7|     126.04|Mar-2010| Fully Paid|         credit_card|Credit Card Elimi...|
| 491575|3bae7a88975616641...|  13750.0|    13750.0|36 months|   14.96|     476.39|Mar-2010| Fully Paid|  debt_consolidation|       Personal Loan|
| 491581|55694e52b6a698777...|   8500.0|     8500.0|36 months|   10.62|     276.76|Mar-2010| Fully Paid|  debt_consolidation|Reduce Debt befor...|
| 423260|c666d309594cceae5...|  10000.0|    10000.0|36 months|    7.88|     312.81|Mar-2010| Fully Paid|first as a Borrow...|first as a relati...|
| 488022|cc859b4c14c9eb521...|   8000.0|     8000.0|36 months|    9.88|     257.69|Mar-2010| Fully Paid|  debt_consolidation|pay off credit ca...|
| 491557|dc1eeb967d0041bec...|  12000.0|    12000.0|36 months|   13.48|     407.09|Mar-2010| Fully Paid|               other|        Payoff Easy!|
| 491494|123126e86ba5ecd5d...|  10000.0|    10000.0|36 months|   14.96|     346.47|Mar-2010| Fully Paid|  debt_consolidation|Credit Card Conso...|
| 491487|aeb1851b33df8f924...|  11000.0|    11000.0|36 months|   13.85|     375.15|Mar-2010| Fully Paid|  debt_consolidation|Credit card conso...|
+-------+--------------------+---------+-----------+---------+--------+-----------+--------+-----------+--------------------+--------------------+
only showing top 20 rows

#### Loan repayments

In [25]:
spark.sql("""select id as loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d from newtable""").repartition(1).write \
.option("header",True)\
.format("csv") \
.mode("overwrite") \
.option("path","/user/itv018355/lendingclubproject/raw/loan_repayments_csv") \
.save()

In [26]:
loan_repayments_df = spark.read \
.format("csv") \
.option("inferSchema", "true") \
.option("header", "true") \
.load("/user/itv018355/lendingclubproject/raw/loan_repayments_csv")

In [27]:
loan_repayments_df

+-------+---------------+-------------+------------------+------------------+---------------+------------+------------+
|loan_id|total_rec_prncp|total_rec_int|total_rec_late_fee|       total_pymnt|last_pymnt_amnt|last_pymnt_d|next_pymnt_d|
+-------+---------------+-------------+------------------+------------------+---------------+------------+------------+
|6300587|        12000.0|      1864.87|20.860000009100002|13885.726104168401|         108.83|    Nov-2014|        null|
|6311418|         7125.0|        95.92|               0.0| 7220.924234675899|        7220.52|    Sep-2013|        null|
|6321535|        24000.0|      7418.84|               0.0|          31418.84|       13979.48|    Apr-2016|        null|
|6300526|        24000.0|      2296.28|               0.0|  26296.2757411036|         730.17|    Aug-2016|        null|
|6301469|        18000.0|      9239.79|               0.0|   27239.794818997|         452.24|    Aug-2018|        null|
|6306881|        20000.0|      2469.32|               0.0|  22469.3219387879|       16744.64|    Oct-2014|        null|
|6301579|        12000.0|      2089.07|               0.0|          14089.07|        4189.07|    Oct-2015|        null|
|6291661|        28000.0|     12441.42|               0.0|  40441.4227009823|       10660.91|    Apr-2017|        null|
|6311612|         3629.9|       847.39|               0.0|           4478.25|         179.13|    Sep-2015|        null|
|6311704|        23600.0|       2077.3|               0.0|   25677.297703591|       21445.23|    Apr-2014|        null|
|6301559|        13375.0|      3926.19|               0.0|  17301.1873419706|         480.18|    Aug-2016|        null|
|6301412|        17500.0|      2062.85|               0.0|19562.849989482198|        3748.28|    Jan-2016|        null|
|6331614|        14000.0|      2442.47|               0.0|  16442.4700020926|        4430.47|    Nov-2015|        null|
|6331589|        1458.43|      1100.69|               0.0|           4051.94|         426.57|    Mar-2014|        null|
|6321730|        14000.0|       719.51|               0.0|14719.505688732199|       11239.57|    May-2014|        null|
|6321586|        15000.0|      1929.76|   25.290000000028|  16955.0514482354|       10354.66|    Oct-2014|        null|
|6331533|        10000.0|      2248.02|               0.0|  12248.0212664276|         339.97|    Aug-2016|        null|
|6321666|        6239.23|      1316.72|               0.0|           7819.45|         260.55|    Jan-2016|        null|
|6328050|        10000.0|      2617.64|               0.0|  12617.6399999982|        4937.87|    Jun-2015|        null|
|6305315|         6000.0|       515.17|               0.0|           6515.17|        4733.17|    Jun-2014|        null|
+-------+---------------+-------------+------------------+------------------+---------------+------------+------------+
only showing top 20 rows

#### Loan defaulters

In [ ]:
spark.sql("""select name_sha2 as member_id , delinq_2yrs , delinq_amnt , pub_rec , pub_rec_bankruptcies , inq_last_6mths ,
total_rec_late_fee, mths_since_last_delinq, mths_since_last_record
from newtable""").repartition(1).write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.option("path","/user/itv018355/lendingclubproject/raw/loan_defaulters_data_csv") \
.save()

In [ ]:
loan_defaulters_data = spark.read \
.format("csv") \
.option("inferSchema", "true") \
.option("header", "true") \
.load("/user/itv018355/lendingclubproject/raw/loan_defaulters_data_csv")

In [ ]:
loan_defaulters_data